In [1]:
import numpy as np
import pandas as pd

In [2]:
book_df = pd.read_csv('../Book recommendation system/Books.csv', low_memory=False)
ratings_df = pd.read_csv('../Book recommendation system/Ratings.csv').sample(40000)
user_df = pd.read_csv('../Book recommendation system/Users.csv')
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

In [3]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN',right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace = True)

In [4]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)

In [5]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID', 
                                                          columns='unique_id_book', 
                                                          values='Book-Rating').fillna(0)

In [6]:
users_books_pivot_matrix_df.head()

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,26091,26092,26093,26094,26095,26096,26097,26098,26099,26100
User-ID,,,,,,,,,,,,,,,,,,,,,
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
from scipy.sparse.linalg import svds

NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [9]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

In [10]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 4.21014705e-33,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  2.31745278e-33,  2.89681598e-33],
       [-1.33490058e-36,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -6.55543806e-37, -8.19429757e-37],
       [ 1.50635909e-33,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  7.40928694e-34,  9.26160868e-34],
       ...,
       [-1.56190528e-33,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00, -7.70449107e-34, -9.63061384e-34],
       [ 1.43719317e-34,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  7.09752219e-35,  8.87190274e-35],
       [ 1.93587880e-25,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  7.47432840e-26,  9.34291050e-26]])

In [11]:
def top_cosine_similarity(data, book_id, top_n=10):
    index = book_id 
    book_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

def similar_books(book_user_rating, book_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    book_user_rating[book_user_rating.unique_id_book == book_id]['Book-Title'].values[0]))
    for id in top_indexes + 1:
        print(book_user_rating[book_user_rating.unique_id_book == id]['Book-Title'].values[0])

In [12]:
k = 50
movie_id =25954  
top_n = 3
sliced = Vt.T[:, :k] 

similar_books(book_user_rating, 25954, top_cosine_similarity(sliced, movie_id, top_n))

Recommendations for Basic Tagalog for Foreigners and Non-Tagalogs (Tuttle Language Library): 

You Are SO Cursed!
They Came to Baghdad (Winterbrook Edition)
Cauldron


C:\Users\trish\AppData\Local\Temp\ipykernel_3532\437162856.py:5: RuntimeWarning: invalid value encountered in true_divide
  similarity = np.dot(book_row, data.T) / (magnitude[index] * magnitude)
